In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

# Step 2: Load the Dataset
# dataset_path = '/content/drive/My Drive/relabeled_pawsx_train/train_paws_x_en.tsv'
# dataset_path = '/content/drive/My Drive/relabeled_pawsx_train/paws-x-test.tsv'
# dataset_path = '/content/drive/My Drive/relabeled_pawsx_train/stsbenchmark_train.tsv'
dataset = pd.read_csv(dataset_path, sep='\t')

Mounted at /content/drive


In [ ]:
# Step 3: Set Up and Use the Model
!pip install transformers accelerate huggingface_hub bitsandbytes

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the model and tokenizer
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

# Convert the model to 4-bit precision
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # Can be changed based on your preference and available resources
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from tqdm import tqdm
from collections import Counter
# Step 6: Define Function to Create Prompts with Sentence Pairs
def create_prompts_with_sentence_pairs(df, system_message):
    prompts = []
    for _, row in df.iterrows():
        prompt = f"""<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
{system_message}<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Are the following sentences paraphrases?


Sentence 1: {row['sentence1']}
Sentence 2: {row['sentence2']}

Answer with 'Yes' or 'No'<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

"""
        prompts.append(prompt)
    return prompts

def generate_batch(prompts, batch_size):
    results = []
    texts = []
    full_texts = []
    for i in tqdm(range(0, len(prompts), batch_size)):
        batch = prompts[i:i + batch_size]
        # Ensure batch is a list of strings
        batch = [str(prompt) for prompt in batch]
        generated = text_generator(
            batch,
            max_new_tokens=3,
            do_sample=True,
            top_k=40,
            top_p=0.95
        )
        for prompt, generation in zip(batch, generated):
            new_text = generation[0]['generated_text'][len(prompt):]
            shortened_text = new_text.lower().strip()[:min(3, len(new_text))]
            prediction = shortened_text == "yes"
            full_texts.append(new_text)
            texts.append(shortened_text)
            results.append(prediction)
        print()
        print("Positive Completed " + str(sum(results)))
        print("Total Completed " + str(len(results)))
    return results


In [ ]:
# Create a text generation pipeline
text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [ ]:
prompts = create_prompts_with_sentence_pairs(dataset,  system_message = "You are a helpful Assistant.")

In [ ]:
import logging
from transformers import logging as transformers_logging

# Suppress transformers logging
transformers_logging.set_verbosity_error()

# Suppress general logging
logging.getLogger("transformers").setLevel(logging.ERROR)


In [ ]:
experiment_name = "LLama3 ICL_4 (Ex. Same Content)"
batch_size = 128
predictions = generate_batch(prompts, batch_size)

  2%|▏         | 1/45 [00:30<22:07, 30.17s/it]


Positive Completed 65
Total Completed 128


  4%|▍         | 2/45 [01:00<21:32, 30.07s/it]


Positive Completed 125
Total Completed 256


  7%|▋         | 3/45 [01:30<21:01, 30.04s/it]

...


Positive Completed 3082
Total Completed 5504


 98%|█████████▊| 44/45 [22:19<00:30, 30.27s/it]


Positive Completed 3175
Total Completed 5632


100%|██████████| 45/45 [22:46<00:00, 30.38s/it]


Positive Completed 3226
Total Completed 5749


In [ ]:
dataset[experiment_name] = predictions

In [ ]:
dataset[experiment_name] = dataset[experiment_name].apply(int)
dataset

,Unnamed: 0,split,genre,dataset,year,sid,score,sentence1,sentence2,LLama3 zero-shot (Ex. Same Content),LLama3 zero-shot (Sem Equiv),LLama3 zero-shot (Paraph)
0,0,train,main-captions,MSRvid,2012test,1,5.00,A plane is taking off.,An air plane is taking off.,1,1,1
1,1,train,main-captions,MSRvid,2012test,4,3.80,A man is playing a large flute.,A man is playing a flute.,1,1,1
2,2,train,main-captions,MSRvid,2012test,5,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...,0,1,0
3,3,train,main-captions,MSRvid,2012test,6,2.60,Three men are playing chess.,Two men are playing chess.,0,0,0
4,4,train,main-captions,MSRvid,2012test,9,4.25,A man is playing the cello.,A man seated is playing the cello.,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5744,5744,train,main-news,headlines,2016,1456,0.00,Severe Gales As Storm Clodagh Hits Britain,Merkel pledges NATO solidarity with Latvia,0,0,0
5745,5745,train,main-news,headlines,2016,1465,0.00,Dozens of Egyptians hostages taken by Libyan t...,Egyptian boat crash death toll rises as more b...,0,0,0
5746,5746,train,main-news,headlines,2016,1466,0.00,President heading to Bahrain,President Xi: China to continue help to fight ...,0,0,0
5747,5747,train,main-news,headlines,2016,1470,0.00,"China, India vow to further bilateral ties",China Scrambles to Reassure Jittery Stock Traders,0,0,0


In [ ]:
dataset.to_csv(dataset_path, sep="\t")